# Pinyin Syllable Count Generator Working Draft Notes
This document details the process of experimentation related to the construction of the Pinyin Syllable Count Generator module. The module takes two columns of string data, one with unboken text in Chinese, the other with its pinyin counterpart. The package will evaluate the number of syllables in the chinese text based on the number of syllables it can detect in the supplied Latin characters. The results will be delivered as integers with commas separating the syllables in each individual word.

The Pandas package will be essential for the vector manipulation and storage.

In [90]:
import pandas as pd
import numpy as np
import csv
import sys

Because pinyin is a structured system with accepted combinations of "initials" (the initial sound of a syllable, e.g. the first consonant) and "finals" (the ending sound of a syllable, e.g. vowels ending with a consonant). One syllable in pinyin can be as small as one letter and as large as six letters. Therefore, detecting a syllable can occur in one of two ways:
1. A sequence of pinyin letters can be compared to a list of accepted pinyin syllables. If the sequence appears in the list, or if the sequence is part of a longer accepted term ("tia" by itself is not an accepted syllable, however "tiao" and "tian" are), then an additional letter from the evaluated word can be added to the sequence and reevaluated. If the sequence does not appear in the list and is not part of a longer accepted term, then the last character of the sequence becomes the first character of the next syllable and the syllable count is incremented.
2. A sequence of pinyin letters is evaluated against a DataFrame of initials and finals, utilizing the .loc command to spot matches if they exist or null values if they don't.

Given these possiblities two metrics need to be considered in order to settle on the proper method for the evaluation function: speed and size. Comparing a sequence to a list of terms can be faster, however the list itself may be excessively large. Evaluating sequences by their combination of initials and finals, however, may take longer but will be result in a much smaller package. The following exercises before will test each method in a smaller scale to get a general sense of the speed vs. size issue.

## Setup for small tests

An example sequence of letters will be tested with both methods.

In [126]:
## LIST ##
# Import list of all pinyin combinations
smList = []
with open("pinyinList.csv") as f:
    r = csv.reader(f)
    for row in r:
        smList += row
        
## NUMPY/DATAFRAME ##
# List of initials, finals for Numpy evaluation
initList = ['ø','b','p','m','f','d','t','n','l','z','c','s',
            'zh','ch','sh','r','j','q','x','g','k','h','y','w']
finList = ['a','ai','ao','an','ang','e','ei','en','eng','er',
           'o','ong','ou','i','ia','iao','ie','iu','ian','iang',
           'in','ing','iong','u','ua','uo','ue','ui','uai','uan','uen','uang','un','v','ve']

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

# Import dataframe from CSV 
smDF = pd.read_csv('pinyinDF.csv', header=0, index_col=0, dtype={'INDEX':str})
smDF = smDF.fillna(0).astype('bool')

print('Python List:')
print(str(smList[50:55]) + '\n')
print('Numpy Array:')
print(str(smAr[0])  + '\n')
print('Pandas DataFrame:')
print(smDF.head(3))

Python List:
['bu', 'pa', 'pai', 'pao', 'pan']

Numpy Array:
[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True False
 False False False False False False False False False False]

Pandas DataFrame:
      a    ai    ao    an   ang      e    ei    en   eng     er  ...     ua  \
ø  True  True  True  True  True   True  True  True  True   True  ...  False   
b  True  True  True  True  True  False  True  True  True  False  ...  False   
p  True  True  True  True  True  False  True  True  True  False  ...  False   

      uo     ue     ui    uai    uan   uang     un      v     ve  
ø  False  False  False  False  False  False  False  False  False  
b  False  False  False  False  False  False  False  False  False  
p  False  False  False  False  False  False  False  False  False  

[3 rows x 34 columns]


In [123]:
smEval = 'teng'
smInitial = 't'
smFinal = 'eng'

# Method 1: Python List
print('Python List: ' + str(sys.getsizeof(smList)))
%timeit -n 100000 smEval in smList

# Method 2: Numpy Array
print('\nNumpy array: ' + str(sys.getsizeof(initList) +
                              sys.getsizeof(finList) +
                              sys.getsizeof(smAr)) +
      ', without initials and finals lists: ' + str(sys.getsizeof(smAr)))
%timeit -n 100000 smAr[initList.index(smInitial),finList.index(smFinal)] != True

# Method 3: Pandas DataFrame
# 3a: slice
print('\nDataFrame: ' + str(sys.getsizeof(smDF)))
print('slice')
%timeit -n 100000 smDF[smFinal][smInitial] != True

# 3b: loc
print('loc')
%timeit -n 100000 smDF.loc[smInitial,smFinal] != True

print('iloc')
# 3c: iloc
%timeit -n 100000 smDF.iloc[initList.index(smInitial),finList.index(smFinal)] != True

Python List: 3760
1.31 µs ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy array: 1528, without initials and finals lists: 928
1.51 µs ± 42.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

DataFrame: 2419
slice
9.74 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
loc
8.12 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
iloc
9.38 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [205]:
import numpy as np
import csv

# List of initials, finals for Numpy evaluation
initList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str, usecols=0)[1:])
finList = list(np.genfromtxt('pinyinDF.csv', delimiter=',',dtype=str)[0][1:])

# Import Numpy array from CSV
smAr = np.genfromtxt('pinyinDF.csv', delimiter=',', skip_header=1, filling_values=0
                    ).astype('bool_')
smAr = np.delete(smAr, 0, 1)

def sylCount(pinyin, test=False):
    '''Count the number of Chinese characters based on inputted pinyin text.'''
    
    # Lists for reference
    vowel = ['a','e','i','o','u','v']
    
    # Variables used for pinyin evaluation
    # Replace ideographic spaces with regular space, also trim leading and following spaces.
    pinyin = pinyin.replace('\u3000',' ').strip()
    
    # Make sure no symbols get through. Throw out null value if so.
    if not pinyin.replace(' ','').isalpha():
        return [0]
    
    # CBDB testing purposes ONLY: if the family name is duplicated in the given name
    dupTest = pinyin.split()
    if len(dupTest) > 1:
        dupTestLen = [len(dupTest[0]), len(dupTest[1])]
        if dupTest[0] == dupTest[1][0:dupTestLen[0]] and dupTestLen[1] > dupTestLen[0]:
            return [0]

    initial = ''
    final = ''
    newSyl = False
    syl = 1
    sylCount = []
    exFin = False
    
    if test == True: print('Testing ' + pinyin)
    
    for i,n in enumerate(str.lower(pinyin)):

        # If current character is a space...
        if n == ' ':
            step = '0. space'
            if pinyin[i+1] == ' ':
                notes = 'discard extra space'
            else:
                sylCount.append(syl)
                newSyl = True
                notes = 'new word'

        # If final has already been set, regardless of current character
        elif final:
            step = 'b. ' + initial + '|' + final + n
            if (final[-1] == 'e' and n == 'r') or (final[-1] == 'n' and n == 'g'):
                notes = final[-1] + n
                if i != len(pinyin) - 1 and pinyin[i+1] != ' ':
                    newSyl = True
                    exFin = final[-1] + n
                    notes += ' - not \"' + exFin + '\" final'
                else:
                    if notes == 'er' and len(final) > 1:
                        syl += 1
                        notes += ' - extra syl at end'

            else:
                notes = 'fin'
                try:
                    initIdx = initList.index(initial)
                    finIdx = finList.index(final + n)
                    result = smAr[initIdx,finIdx]
                    notes += ' - trying ' + initial + final + n + ': ' + str(result)
                except:
                    result = False
                    notes += ' - array error'

                if not result:
                    listFins = list(i for i in finList if i.startswith(final+n))
                    if not listFins or not any(i for i in listFins if smAr[initList.index(initial),finList.index(i)]):
                        newSyl = True
                        notes += ' - new syl'
                    else:
                        final += n
                        notes += ' - definitely more'
                else:
                    final += n
                    notes += ' - maybe more'
          
        # If there is no final set (work on initial)
        else:
            step = 'a. ' + initial + n
            notes = 'init'
            if n not in vowel:
                if initial and n != 'h':
                    if test == True: print(initial + n + ' - quitting, two initials')
                    return [0]
                initial += n
                notes += ' - cons'
            elif not initial:
                initial = 'ø'
                final = n
                notes += ' - starting vowel'
            else:
                final = n
                notes += ' - vowel'
                
        # Print debug notes if parameter is true
        if test == True: print(step + ': ' + notes)
        
        # Processes for next syllable
        if newSyl:
            if n in vowel:
                initial = 'ø'
                final = n
                syl += 1
            elif n != ' ':
                if exFin and pinyin[i+1] not in vowel:
                    initial = ''
                    if exFin == 'er' and len(final) > 1:
                        syl+= 1
                else:
                    initial = n
                exFin = False
                final = ''
                syl += 1
            else:
                initial = ''
                final = ''
                syl = 1
            if test == True: print('new syl: ' + str(syl))
            newSyl = False
    
    sylCount.append(syl)
    
    return sylCount

In [219]:
import pandas as pd

# Import CBDB data
names = pd.read_csv('Chinese Names.csv')

# Run sylCount function on 'EngName' column
names['syl'] = names['EngName'].apply(sylCount, test=False)

# Add 'check' column to validate results
names['check'] = names['ChiName'].str.len() == names['syl'].apply(sum)

# Filter results to show false negatives
names[~names['check']][names['syl'].apply(sum) > 0][~names['ChiName'].str.contains('\(')]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


,EngName,ChiName,syl,check
433,Aixinjueluo Bukuliyongshun,布庫里雍順,"[4, 5]",False
455,Aixinjueluo Chongshan,充善,"[4, 2]",False
500,Aixinjueluo Duoergun,多尔衮,"[4, 3]",False
504,Aixinjueluo Eerdeng,愛新覺羅爾登,"[4, 3]",False
534,Aixinjueluo Fulin,福临,"[4, 2]",False
535,Aixinjueluo Fuman,福滿,"[4, 2]",False
592,Aixinjueluo Hongli,弘历,"[4, 2]",False
615,Aixinjueluo Huangtaiji,皇太极,"[4, 3]",False
648,Aixinjueluo Jishen,愛新覺羅紳,"[4, 2]",False
649,Aixinjueluo Juechangan,覺昌安,"[4, 3]",False


In [220]:
len(names)

65534

In [221]:
sylCount('Bao Xianian',True)

Testing Bao Xianian
a. b: init - cons
a. ba: init - vowel
b. b|ao: fin - trying bao: True - maybe more
0. space: new word
new syl: 1
a. x: init - cons
a. xi: init - vowel
b. x|ia: fin - trying xia: True - maybe more
b. x|ian: fin - trying xian: True - maybe more
b. x|iani: fin - array error - new syl
new syl: 2
b. ø|ia: fin - trying øia: False - new syl
new syl: 3
b. ø|an: fin - trying øan: True - maybe more


[1, 3]

In [238]:
countAccuracy = (round(len(names[~names['check']][names['syl'].apply(sum) > 0][~names['ChiName'].str.contains('\(')]) / len(names) * 100, 2) - 100) * -1
print(str(countAccuracy) + '%')

99.63%


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
